Notebook referente a [issue][issue]

[issue]: https://github.com/sepep-pmsp/siiau/issues/33

In [ ]:
import requests
from pathlib import Path
import zipfile
import pandas as pd
import concurrent.futures
import geopandas as gpd
from shapely import Point
from IPython.display import clear_output

In [ ]:
!pip install basedosdados==2.0.0b16
clear_output()

# Funçoes

In [ ]:
def download_and_zip_extraction(url: str) -> None:

    base_path = Path("data")
    path_input = base_path / "input"

    response = requests.get(url, stream=True)
    response.raise_for_status()

    save_path = path_input / url.split('/')[-1]

    path_input.mkdir(parents=True, exist_ok=True)

    with open(save_path, "wb") as fd:
        for chunk in response.iter_content(chunk_size=128):
            fd.write(chunk)

    with zipfile.ZipFile(save_path) as z:
        z.extractall(path_input)

# Querys e constantes

In [ ]:
query_ms_sinasc = """
SELECT
    *
FROM `basedosdados.br_ms_sinasc.microdados` AS dados
where ano = 2023 and id_municipio_nascimento = '3550308' and codigo_estabelecimento is not null
"""

query_ms_sim = """
SELECT
  *
FROM `basedosdados.br_ms_sim.microdados` AS dados
where ano = 2022 and id_municipio_ocorrencia = '3550308' and codigo_estabelecimento is not null
"""

billing_project_id = "br-fbsp-absp" # Define o nome do projeto Google que está vinculado ao email do colab

# Download [CNES][cnes] Estabelecimentos e [Censo][censo] 2020 geometry

[cnes]: https://dados.gov.br/dados/conjuntos-dados/cnes-cadastro-nacional-de-estabelecimentos-de-saude
[censo]: http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP

In [ ]:
url = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/CNES/cnes_estabelecimentos.zip'
url_sp_distrito_geo = 'http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP/SP_distritos_CD2022.zip'

urls = [url, url_sp_distrito_geo]

with concurrent.futures.ThreadPoolExecutor() as executor:
  executor.map(download_and_zip_extraction, urls)

# Ler [CNES][cnes] estabelecimentos

[cnes]: https://dados.gov.br/dados/conjuntos-dados/cnes-cadastro-nacional-de-estabelecimentos-de-saude

In [ ]:
cnes_estabelecimentos = pd.read_csv('/content/data/input/cnes_estabelecimentos.csv', dtype=str, sep=';',
                                    encoding='latin-1', usecols=['CO_CNES', 'CO_UNIDADE', 'CO_IBGE', 'NU_LATITUDE', 'NU_LONGITUDE'])

cnes_estabelecimentos_sp = cnes_estabelecimentos[cnes_estabelecimentos.CO_IBGE == '355030']

# Transformar longitue e latitude em Point objeto
cnes_estabelecimentos_sp['point'] = cnes_estabelecimentos_sp.apply(lambda row: Point(row.NU_LONGITUDE, row.NU_LATITUDE), axis=1)

cnes_estabelecimentos_sp_geo_df = gpd.GeoDataFrame(cnes_estabelecimentos_sp, geometry='point')

cnes_estabelecimentos_sp_geo_df.crs = 'EPSG:4674' # igular CRS

# Ler geometry do [Censo 2022][censo]

[censo]: http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP

In [ ]:
geo_df_sp = gpd.read_file('/content/data/input/SP_distritos_CD2022.shp', columns=['CD_DIST', 'NM_DIST', 'geometry'])
geo_df_sp = geo_df_sp[geo_df_sp.CD_DIST.str.contains("3550308")]

# Juntas [CENSO][censo] Geometry com Point de [CNES][cnes]

[cnes]: https://dados.gov.br/dados/conjuntos-dados/cnes-cadastro-nacional-de-estabelecimentos-de-saude
[censo]: http://ftp.ibge.gov.br/Censos/Censo_Demografico_2022/Agregados_por_Setores_Censitarios/malha_com_atributos/distritos/shp/UF/SP

In [ ]:
cnes_sp_distrito_geo_df = geo_df_sp.sjoin(cnes_estabelecimentos_sp_geo_df, how='left', predicate='contains')

# Puxar dados [Sinasc][sinasc]

[sinasc]: https://basedosdados.org/dataset/48ccef51-8207-40ee-af5b-134c8ac3fb8c?table=80359f9a-8189-4693-bdf7-ebf7be0d2fff

In [ ]:
import basedosdados as bd

df_ms_sinasc = bd.read_sql(
      query_ms_sinasc,
      billing_project_id=billing_project_id,
  )

df_ms_sinasc.codigo_estabelecimento = df_ms_sinasc.codigo_estabelecimento.str.lstrip('0') # Limpando zero a esquerda

Downloading: 100%|██████████|


# Juntar [SINASC][sinasc] com [CNES][cnes] com distritos

[sinasc]: https://basedosdados.org/dataset/48ccef51-8207-40ee-af5b-134c8ac3fb8c?table=80359f9a-8189-4693-bdf7-ebf7be0d2fff
[cnes]: https://dados.gov.br/dados/conjuntos-dados/cnes-cadastro-nacional-de-estabelecimentos-de-saude

In [ ]:
df_ms_sinasc_with_distritos = df_ms_sinasc.merge(cnes_sp_distrito_geo_df, how='left', left_on='codigo_estabelecimento', right_on='CO_CNES')

# Indicadores vindo de [SINASC][sinasc]

[sinasc]: https://basedosdados.org/dataset/48ccef51-8207-40ee-af5b-134c8ac3fb8c?table=80359f9a-8189-4693-bdf7-ebf7be0d2fff

### Proporção de nascidos vivos de mães que fizeram 7 ou mais consultas de pré natal

In [ ]:
df_ms_sinasc_pre_natal = df_ms_sinasc_with_distritos[df_ms_sinasc_with_distritos.pre_natal >= 7]
df_ms_sinasc_pre_natal = df_ms_sinasc_pre_natal.groupby('NM_DIST').pre_natal.count()

### Proporção de bebês nascidos vivos de mães adolescentes (10 a 19 anos)

In [ ]:
maes_adolescentes_mark = (df_ms_sinasc_with_distritos.idade_mae <= 19) & (df_ms_sinasc_with_distritos.idade_mae >= 10)
df_ms_sinasc_maes_adolescentes = df_ms_sinasc_with_distritos[maes_adolescentes_mark]
df_ms_sinasc_maes_adolescentes = df_ms_sinasc_maes_adolescentes.groupby('NM_DIST').idade_mae.count()

### Proporção de bebês com baixo peso ao nascer (menos de 2,5 kg)

In [ ]:
bebes_baixo_peso_mark = (df_ms_sinasc_with_distritos.peso <= 2500)
df_ms_sinasc_bebes_baixo_peso = df_ms_sinasc_with_distritos[bebes_baixo_peso_mark]
df_ms_sinasc_bebes_baixo_peso = df_ms_sinasc_bebes_baixo_peso.groupby('NM_DIST').peso.count()

# Puxar dados [ms_sim][ms_sim] Microdados

[ms_sim]: https://basedosdados.org/dataset/5beeec93-cbf3-43f6-9eea-9bee6a0d1683?table=dea823a5-cad7-4014-b77c-4aa33b3b0541

In [ ]:
df_ms_sim = bd.read_sql(
      query_ms_sim,
      billing_project_id=billing_project_id,
  )

df_ms_sim.codigo_estabelecimento = df_ms_sim.codigo_estabelecimento.str.rstrip('.0') # Limpando zero a esquerda

Downloading: 100%|██████████|


# Juntar [MS_SIM][ms_sim] com [CNES][cnes] com distritos

[ms_sim]: https://basedosdados.org/dataset/5beeec93-cbf3-43f6-9eea-9bee6a0d1683?table=dea823a5-cad7-4014-b77c-4aa33b3b0541
[cnes]: https://dados.gov.br/dados/conjuntos-dados/cnes-cadastro-nacional-de-estabelecimentos-de-saude

In [ ]:
df_ms_sim_with_distritos =  df_ms_sim.merge(cnes_sp_distrito_geo_df, how='left', left_on='codigo_estabelecimento', right_on='CO_CNES')

# Indicadores vindo de [MS_SIM][ms_sim]

[ms_sim]: https://basedosdados.org/dataset/5beeec93-cbf3-43f6-9eea-9bee6a0d1683?table=dea823a5-cad7-4014-b77c-4aa33b3b0541

### Mortalidade materna

In [ ]:
df_ms_sim_with_mortalidade_materna = df_ms_sim_with_distritos.groupby('NM_DIST').data_obito.count()

### Idade média ao morrer

In [ ]:
df_ms_sim_with_idade_media = df_ms_sim_with_distritos.groupby('NM_DIST').idade.mean().round(2)

# Juntando indicadores -> [Excel][excel]

[excel]: https://docs.google.com/spreadsheets/d/1wDRZC8e1xuYIk99AHdjL5LkIFevf_6HU/edit?usp=sharing&ouid=115854210799010302684&rtpof=true&sd=true




In [ ]:
colunas = [
    'Proporção de nascidos vivos de mães que fizeram 7 ou mais consultas de pré natal - SINASC 2023',
    'Proporção de bebês nascidos vivos de mães adolescentes (10 a 19 anos) - SINASC 2023',
    'Proporção de bebês com baixo peso ao nascer (menos de 2,5 kg) - SINASC 2022',
    'Mortalidade materna - SIM 2022',
    'Idade média ao morrer - SIM 2022']

indicadores_sinasc = [df_ms_sinasc_pre_natal,
                      df_ms_sinasc_maes_adolescentes,
                      df_ms_sinasc_bebes_baixo_peso,
                      df_ms_sim_with_mortalidade_materna,
                      df_ms_sim_with_idade_media]

indicadores_sinasc = pd.concat(indicadores_sinasc, axis=1)

indicadores_sinasc.columns = colunas
indicadores_sinasc.index.names = ['Distritos']

indicadores_sinasc.to_excel('indicadores_mortalidade.xlsx')